In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
import glob
import PIL, cv2
import os
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../scripts/')
from swin_transformer import final_model
%load_ext autoreload
%autoreload 2
import tensorflow.keras.backend as K

2023-07-27 21:48:32.719714: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 21:48:32.885466: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-27 21:48:33.622805: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64
2023-07-27 21:48:33.622915: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: can

In [2]:
allowed_gpus = [0]
gpus = tf.config.list_physical_devices('GPU')
final_gpu_list = [gpus[x] for x in allowed_gpus]
tf.config.set_visible_devices(final_gpu_list, 'GPU')
for gpu in final_gpu_list:
    tf.config.experimental.set_memory_growth(gpu, True)
strategy = tf.distribute.MirroredStrategy()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

2023-07-27 21:48:35.359898: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-27 21:48:35.403002: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-27 21:48:35.403356: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-27 21:48:35.404996: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


 SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-27 21:48:36.154704: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7181 MB memory:  -> device: 0, name: Quadro P4000, pci bus id: 0000:00:05.0, compute capability: 6.1


In [3]:
csv_file = pd.read_csv('../../files/train.csv')

In [4]:
csv_file.head()

,image_id,center_id,patient_id,image_num,label
0,006388_0,11,006388,0,CE
1,008e5c_0,11,008e5c,0,CE
2,00c058_0,11,00c058,0,LAA
3,01adc5_0,11,01adc5,0,LAA
4,026c97_0,4,026c97,0,CE


In [5]:
csv_file['fixed_paths'] = csv_file['image_id'].apply(lambda x: '../../files/resized_train/' + x + '.png')

In [6]:
csv_file.head()

,image_id,center_id,patient_id,image_num,label,fixed_paths
0,006388_0,11,006388,0,CE,../../files/resized_train/006388_0.png
1,008e5c_0,11,008e5c,0,CE,../../files/resized_train/008e5c_0.png
2,00c058_0,11,00c058,0,LAA,../../files/resized_train/00c058_0.png
3,01adc5_0,11,01adc5,0,LAA,../../files/resized_train/01adc5_0.png
4,026c97_0,4,026c97,0,CE,../../files/resized_train/026c97_0.png


In [7]:
for x in csv_file['fixed_paths'].values:
    if os.path.exists(x) == False:
        print(x)

In [8]:
def split_datasets(csv_file, test_size = 0.01):
    train, test = train_test_split(csv_file, test_size=test_size)
    train, val = train_test_split(train, test_size=test_size)
    return train, val, test

In [9]:
def read_imgs(img, label):
    img = tf.io.read_file(img)
    img = tf.image.decode_png(img, channels=3)
    img = img / 255
    return img, label

In [10]:
def get_data(csv_file, repeat=True, shuffle=True, batch=True, batch_size=16):
    imgs, labels = csv_file['fixed_paths'].values.tolist(), [1 if x == 'CE' else 0 for x in csv_file['label'].values.tolist()]
    tensor = tf.data.Dataset.from_tensor_slices((imgs, labels))
    tensor = tensor.cache()
    if repeat:
        tensor = tensor.repeat()
    if shuffle:
        tensor = tensor.shuffle(256 * REPLICAS)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        tensor = tensor.with_options(opt)
    tensor = tensor.map(read_imgs, num_parallel_calls=AUTO)
    if batch:
        tensor = tensor.batch(batch_size * REPLICAS)
    tensor = tensor.prefetch(AUTO)
    return tensor

In [11]:
train, val, test = split_datasets(csv_file, test_size=0.05)

In [12]:
train_dataset = get_data(train)
val_dataset = get_data(val)
test_dataset = get_data(test)

In [13]:
input_shape = (256, 256, 3)
patch_size = (2,2)
num_patch_x = input_shape[0] // patch_size[0]
num_patch_y = input_shape[1] // patch_size[1]
embed_dim = 64
num_heads = 8
window_size = 2
dropout_rate = 0.0
qkv_bias = True
num_mlp = 256
shift_size = 1

In [14]:
test_arr = np.random.random((10, 256, 256, 3))

In [15]:
with strategy.scope():
    model = final_model(input_shape, patch_size, embed_dim, num_heads, window_size, dropout_rate, qkv_bias, num_mlp, shift_size)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [16]:
K.clear_session()
# log_dir = f"{os.environ['tb_path']}segmentation/res50_baseline_bs_128_is_32/"
# if os.path.exists(log_dir) == False:
#     os.makedirs(log_dir)
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
# weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/segmentation/res50_baseline_bs_128_is_32/'
# weights_save = CallbackForSavingModelWeights(weights_path)
batch_size = 4
train_dataset = get_data(train, batch_size=batch_size)
val_dataset = get_data(val, shuffle=False, repeat=False, batch_size=batch_size)
# model_config_file
input_shape = (256, 256, 3)
patch_size = (2, 2)
num_patch_x = input_shape[0] // patch_size[0]
num_patch_y = input_shape[1] // patch_size[1]
embed_dim = 128
num_heads = 16
window_size = 2
dropout_rate = 0.0
qkv_bias = False
num_mlp = 1024
shift_size = 1
# model traning loop
with strategy.scope():
    model = final_model(input_shape, patch_size, embed_dim, num_heads, window_size, dropout_rate, qkv_bias, num_mlp, shift_size)
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 100,
        verbose = 1,
        validation_data = val_dataset,
        # callbacks = [tensorboard_callback]
    )

Epoch 1/100


2023-07-27 21:48:41.628145: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 680
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\025TensorSliceDataset:24"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
      shape {
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
     

170/170 [==============================] - ETA: 0s - loss: 1.2259 - accuracy: 0.0382 - tp: 415.0000 - fp: 160.0000

2023-07-27 21:49:45.701793: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 36
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\025TensorSliceDataset:32"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
      shape {
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
      

170/170 [==============================] - 67s 361ms/step - loss: 1.2259 - accuracy: 0.0382 - tp: 415.0000 - fp: 160.0000 - val_loss: 0.5999 - val_accuracy: 0.0000e+00 - val_tp: 26.0000 - val_fp: 10.0000
Epoch 2/100
170/170 [==============================] - 60s 352ms/step - loss: 0.6655 - accuracy: 0.0000e+00 - tp: 451.0000 - fp: 162.0000 - val_loss: 0.8421 - val_accuracy: 0.0000e+00 - val_tp: 26.0000 - val_fp: 10.0000
Epoch 3/100
170/170 [==============================] - 60s 353ms/step - loss: 0.6902 - accuracy: 0.0000e+00 - tp: 439.0000 - fp: 180.0000 - val_loss: 0.6496 - val_accuracy: 0.0000e+00 - val_tp: 26.0000 - val_fp: 10.0000
Epoch 4/100
170/170 [==============================] - 60s 353ms/step - loss: 0.6758 - accuracy: 0.0000e+00 - tp: 438.0000 - fp: 175.0000 - val_loss: 0.5970 - val_accuracy: 0.0000e+00 - val_tp: 26.0000 - val_fp: 10.0000
Epoch 5/100
170/170 [==============================] - 60s 352ms/step - loss: 0.6265 - accuracy: 0.0000e+00 - tp: 480.0000 - fp: 181.000